# Peavey VIP-2 USB Scanner

![Peavey VIP-2](vip2-small.png)

Other relevant documentation includes:

* Peavey VIP-[1,2,3] manual https://assets.peavey.com/literature/manuals/118347_18396.pdf
* Peavey Sanpura Footswitch manual https://assets.peavey.com/literature/manuals/118366_34856.pdf
* Peavey Vypyr Pro MIDI Specification (not entirely the same, but ideas are shared) https://assets.peavey.com/literature/additional/118745_31036.pdf
* My own footswitch controller for Arduino https://github.com/aughey/peavey_footswitch/blob/master/peavey_footswitch.ino
* Another DIY footswitch http://www.claytonfelt.com/peavey-vypyr-vip-amp-diy-midi-footswitch/
* Link to a reverse engineered MIDI spec for the commercial footswitch https://www.vguitarforums.com/smf/index.php?topic=3119.0

The trick here is to reverse engineer data captured from wireshark of the Peavey supplied Windows application.  The relevant
capture is below.

![Wireshark](wireshark.PNG)

This is the message sent by the application to retrieve the current amp state.  In response to a preset change, the
app sends this syex message and the amp response with a long sysex string.

Unfortunately, Wireshark is displaying this USB capture strangely.  It's interjecting 0x04 values every 4 values, so
the capture data 0x04 0xf0 0x00 0x00 0x04 0x1b 0x12 0x00 0x04, with the 0x04's stripped out is really, 0xf0, 0x00, 0x00, 0x1b, 0x12, 0x00.  I can't explain why wireshark is showing it this way, but it is.  Strangely, at the end there is 0x05 instead of 0x04 insertted.

In [1]:
import mido
import sys
import time
import json

Get Find the midi ports with VYPYR in in

In [2]:
input = None
output = None

for name in mido.get_output_names():
  if name.startswith("VYPYR"):
    output = mido.open_output(name)
    print("Output Opened: ",name)
    
for name in mido.get_input_names():
  if name.startswith("VYPYR"):
    input = mido.open_input(name)
    print("Input Opened:  ",name)


Output Opened:  VYPYR USB Interface:VYPYR USB Interface MIDI 1 20:0
Input Opened:   VYPYR USB Interface:VYPYR USB Interface MIDI 1 20:0


## Methods for manipulating the device over MIDI

In [3]:
prefix = [ 0x00, 0x00, 0x1B, 0x12, 0x00]
def send_control(id,value):
    output.send(msg = mido.Message("control_change", control=id, value=value))

def soft_reset():
    msg = mido.Message('sysex', data= prefix + [0x0c,0x0,0x55])
    output.send(msg)

def factory_reset():
    msg = mido.Message('sysex', data= prefix + [0x0c,8,0x55])
    output.send(msg)
    
def send_query():
    flush()
    msg = mido.Message('sysex', data= prefix + [0x63, 0x7f, 0x7f])
    output.send(msg)
    
def flush():
    for msg in input.iter_pending():
        None
        
def read_data():
    for msg in input.iter_pending():
       print(msg)
       print(msg.hex())
        
def read_block(type = None):
    for msg in input:
       if type == None or type == msg.type:
           return msg
        
def read_conf():
    send_query();
    return read_block(type='sysex')

def wait_for_data(t):
    start = time.time()
    while time.time() - start < t:
        msg = input.poll()
        print("a")
        if msg:
            return msg
    return None

read_data()
flush()

In [4]:
read_conf()

<message sysex data=(0,0,27,18,9,99,127,127,0,15,2,15,2,5,3,14,7,15,5,15,2,5,0,6,0,6,0,10,0,0,3,7,3,7,1,15,2,8,2,5,0,0,0,0,4,0,4,0,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,0,0,0,0,0,0,0,0,4,12,4,5,4,1,4,4,2,0,4,7,4,1,4,9,4,14,2,0,2,0,2,0,2,0,2,0,2,0,2,0,2,0,2,0,2,0,2,0,2,0,2,0,2,0,2,0,2,0,2,0,2,0,2,0,2,0,2,0,2,0,2,0,10,5) time=0>

Mostly reference here for how to 

In [5]:
msg = mido.Message('note_on',note=0x10, channel=1)
output.send(msg);
msg = mido.Message('note_off',note=0x10, channel=1)
output.send(msg);


If we really need to write raw data, the _rt method is available, but thankfully this sysex approach is going to work.

In [6]:
output._rt

## Inquiry command - Get version

This sysex message is documented in the Vypyr Pro manual as an Inquiry command.  

The response to a standard inquiry is: F0 7E 7F 06 02 00 00 1B 30 00 00 00 ww xx yy zz F7

where:

'ww' is 41 ('A' amp mode), 44 ('D' demo mode), 54 ('T' tuner mode) or 42 ('B' bootcode -
software update mode)

'xx' is the major version number (30 or 31 for 0.0 or 1.0)

'yy' and 'zz' are the minor version digits (30..39 for a range of .00 to .99)

Perhaps this can be used to query the type of amp before sending other commands to check for compatibility.

In [7]:
#msg = mido.Message('sysex', data=[0x00,0x00,0x04,0x1b,0x12,0x00,0x04,0x63,0x7f,0x7f,0x05])
flush()
msg = mido.Message('sysex', data=[ 0x7E, 0x7F, 0x06, 0x01])
print(msg.hex())

F0 7E 7F 06 01 F7


In [8]:
output.send(msg)
inquiry_reply = read_block()
print(inquiry_reply.hex());

F0 7E 00 06 02 00 00 1B 12 00 09 00 30 31 36 32 F7


In [9]:
major = 0
wwxxyyzz = inquiry_reply.data[11:]
if wwxxyyzz[1] == 0x31:
    major = 1
minor = (wwxxyyzz[2] - 0x30) + (wwxxyyzz[3] - 0x30) * 10
print("Version: " + str(major) + "." + str(minor))

Version: 1.26


# Read the configuration block

In [10]:
send_query()
print(read_block().hex())

F0 00 00 1B 12 09 63 7F 7F 00 0F 02 0F 02 05 03 0E 07 0F 05 0F 02 05 00 06 00 06 00 0A 00 00 03 07 03 07 01 0F 02 08 02 05 00 00 00 00 04 00 04 00 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 0F 00 00 00 00 00 00 00 00 04 0C 04 05 04 01 04 04 02 00 04 07 04 01 04 09 04 0E 02 00 02 00 02 00 02 00 02 00 02 00 02 00 02 00 02 00 02 00 02 00 02 00 02 00 02 00 02 00 02 00 02 00 02 00 02 00 02 00 02 00 02 00 02 00 0A 05 F7


## The result of this should be true.  Just reading the block twice

In [11]:
send_query()
a = read_block()
time.sleep(1)
send_query()
b = read_block()
a == b

True

In [12]:
def what_changed(a,b,doprint=True):
    i=0
    count = 0
    changed = []
    myindex = None
    for v in zip(a.data,b.data):
        if v[0] != v[1]:
            if doprint:
                print("  Index: ",i," ",hex(v[0])," != ",hex(v[1]))
            if v[0] == 1 and v[1] == 2:
                myindex = i
            changed.append(i)
            count += 1
        i=i+1
    return { 'myindex': myindex, 'changed': changed, 'count': count }

In [13]:
flush()
a = read_conf()

In [14]:
b = read_conf()
what_changed(a,b)

{'changed': [], 'count': 0, 'myindex': None}

In [15]:
for c in range(0,20):
    send_control(c,2)
flush()

We find a control by setting the value of the control to 1, reading it, changing the control value to 2, and reading
the configuration a second time.  One or more fields should change.  The changed field(s) are the values affected by this
control.

In [25]:
def find_control(id):
    send_control(id,1);
    a = read_conf()
    send_control(id,2);
    b = read_conf()
    return what_changed(a,b)

In [34]:
flush()
find_control(20)

  Index:  19   0x1  !=  0x2


{'changed': [19], 'count': 1, 'myindex': 19}

In [35]:
find_control(21)

  Index:  35   0x1  !=  0x2


{'changed': [35], 'count': 1, 'myindex': 35}

In [36]:
find_control(26)

  Index:  29   0x1  !=  0x2


{'changed': [29], 'count': 1, 'myindex': 29}

## Documentation:

Below are the control values for the knobs on the VIP-2.  These were determined by rotating knobs and watching the MIDI stream.

### Control Change

* pregain = 16
* low = 17
* mid = 18
* high = 19
* postgain = 20

* P1 = 27
* P2 = 26
* Delay Feedback = 21
* Delay Level = 23
* Reverb = 31

* Effect = 10
* Amp = 8
* Inst/Stomp = 11

![Front Panel](frontpanel.png)


In [37]:
known_controls = {
    "amptype" : 12,
    "Pre-Gain" : 16,
    "Low": 17,
    "Mid" : 18,
    "High" : 19,
    "Post-Gain" : 20,
    "P1" : 27,
    "P2" : 26,
    "SB_P1" : 25,
    "SB_P2" : 24,
    "Delay Feedback" : 21,
    "Delay Level" : 23,
    "Reverb" : 31,
    "Effect" : 10,
    "Amp" : 8,
    "stompbox" : 11
}

# Loop through all controls from 1 to 32 inclusive

Print out the configuration indices that change.  The controls that change should correspond to the known controls above.

More importantly, this searches for controls that are represented in the configuration that we do not have in our known
controls list.  Any control that hits this warning should be checked to see what it affects.

In [38]:
#send_control(known_controls['Amp'],1) # Reset amp to 0
#send_control(known_controls['Effect'],1) # Reset effect to 1
#send_control(known_controls['stompbox'],13) # Reset stompbox to 13 Comp
new_controls = {}
mapped_indices = {}
flush()
for control,control_id in known_controls.items():
    print("Control: ",control,"(",control_id,")")
    info = find_control(control_id)
    if 0 == info['count']:
        print("  Warning: this is a known control, but we didn't see a change in the configuration")
    new_controls[control] = { 'control': control_id, 'config_index': info['myindex']}
    if not info['myindex']:
        continue
    mapped_indices[info['myindex']] = True
    mapped_indices[info['myindex']-1] = True

Control:  P2 ( 26 )
  Index:  29   0x1  !=  0x2
Control:  Effect ( 10 )
  Index:  25   0x1  !=  0xc
  Index:  26   0x3  !=  0x0
  Index:  27   0x4  !=  0x2
Control:  High ( 19 )
  Index:  15   0x1  !=  0x2
Control:  SB_P1 ( 25 )
  Index:  31   0x1  !=  0x2
Control:  SB_P2 ( 24 )
  Index:  33   0x1  !=  0x2
Control:  stompbox ( 11 )
  Index:  8   0x0  !=  0x1
  Index:  9   0x8  !=  0xa
  Index:  10   0x6  !=  0x0
  Index:  11   0x4  !=  0x2
  Index:  12   0x3  !=  0x0
  Index:  13   0xf  !=  0x2
  Index:  14   0x5  !=  0x0
  Index:  15   0x5  !=  0x2
  Index:  17   0x0  !=  0x2
  Index:  18   0x7  !=  0x0
  Index:  19   0xf  !=  0x2
  Index:  43   0x4  !=  0xa
  Index:  44   0x5  !=  0x4
  Index:  45   0x4  !=  0x0
  Index:  46   0x2  !=  0x0
Control:  Pre-Gain ( 16 )
  Index:  17   0x1  !=  0x2
Control:  Amp ( 8 )
  Index:  8   0x2  !=  0x1
  Index:  9   0x3  !=  0xa
  Index:  10   0x6  !=  0x0
  Index:  11   0x4  !=  0x2
  Index:  12   0x3  !=  0x0
  Index:  13   0x5  !=  0x2
  Index:

In [39]:
for control in new_controls:
    data = new_controls[control]
    if data['config_index'] != None:
        continue
    print("No config index for " + control + " (" + str(data['control']) + ")")
    flush()
    id = data['control']
    send_control(id,1);
    a = read_conf()
    send_control(id,2);
    b = read_conf()
    changed = what_changed(a,b,doprint=False)
    for changed_id in changed['changed']:
        if not changed_id in mapped_indices:
            print("  ",changed_id," is not one of the known indices")

No config index for Effect (10)
   25  is not one of the known indices
No config index for stompbox (11)
   8  is not one of the known indices
   9  is not one of the known indices
   43  is not one of the known indices
   44  is not one of the known indices
   45  is not one of the known indices
   46  is not one of the known indices
No config index for Amp (8)
   8  is not one of the known indices
   9  is not one of the known indices
No config index for amptype (12)
   8  is not one of the known indices
   9  is not one of the known indices


In [32]:
# Create permutators
def permutate(permutations):
    if len(permutations) == 0:
        yield []
        return
        
    p = permutations.pop()
    for a in permutate(permutations):
        for me in p:
            a.append(me)
            yield a
            a.pop()
    permutations.append(p)

for p in permutate([range(0,3),range(5,8)]):
    print(p)

[0, 5]
[0, 6]
[0, 7]
[1, 5]
[1, 6]
[1, 7]
[2, 5]
[2, 6]
[2, 7]


# Brute force figure out the mapping for amps

In [ ]:
def brute_force(control_names, value_ranges, config_indices):
    flush()
    value_pair_map = {}
    for p in permutate(value_ranges):
        # Set the controls for this permutation
        print("controls:",control_names,"current p:",p)
        for index,name in enumerate(control_names):
            send_control(known_controls[name],p[index])
        # Read the configuration
        cur = read_conf()
        # Our key is actually the current value of the config parameters
        key = [ str(cur.data[i]) for i in config_indices ]
        # Make a string out of it separated by commas
        key = ",".join(key)
        value = {}
        print(key)
        for index,name in enumerate(control_names):
            value[name] = p[index]
        if key in value_pair_map:
            print(control_names)
            print("Error: Found duplicate key",key)
            print("Permutation: ",p)
        assert key not in value_pair_map
        value_pair_map[key] = value
    return [config_indices,value_pair_map]

send_control(known_controls['Amp'],0) # Reset amp to 0
send_control(known_controls['Effect'],1) # Reset effect to 1
send_control(known_controls['stompbox'],13) # Reset stompbox to 13 Comp
config_mappings = []
#config_mappings =  brute_force(control_names = ["Amp","amptype"], value_ranges = [range(0,12),range(0,3)], config_indices=[8,9])
config_mappings += brute_force(control_names = ["Effect"], value_ranges = [range(0,12)], config_indices=[25])
config_mappings += brute_force(control_names = ["stompbox"], value_ranges = [range(0,20)], config_indices=[43])

In [ ]:
print("// Values determined using python reverse engineering amp")
print("// Do not change this, use the scanner notebook to compute.")
print("var known_controls =",json.dumps(new_controls))
print("var config_mappings =",json.dumps(config_mappings))